In [2]:
import numpy as np
import sys
import trimesh
import tetgen 
import meshio 
import math

import matplotlib.pyplot as plt
import pymeshfix as pfix
import pyvista as pv
import iso2mesh as i2m
import pygalmesh as pygm
import pymeshlab as ml

from skimage.filters import threshold_otsu
from tqdm.notebook import tqdm
from skimage.transform import resize
from typing import Any, Dict, Optional, Tuple, List

sys.path.append('./src')  # to import alveoRVE from parent directory

from alveoRVE.plot.mpl import show_four_panel_volume
from alveoRVE.plot.pv import view_surface

%load_ext autoreload
%autoreload 2

In [3]:
def matlab_to_python_conv(no, fc): 
    no_out = no[:, :3].copy()
    fc_out = (np.atleast_2d(fc).astype(np.int64)[:, :3] - 1).astype(np.int32)
    return no_out, fc_out

In [4]:
def quick_mesh_report(ms: ml.MeshSet | trimesh.Trimesh, i: int = 0):
    # print(f"\n == pymeshlab quick metrics:")
    # number of vertices and faces

    flag = False
    if isinstance(ms, trimesh.Trimesh):
        flag = True
        V, F = ms.vertices, ms.faces
        ms = ml.MeshSet()
        ms.add_mesh(ml.Mesh(V, F))
    else: 
        V = ms.current_mesh().vertex_matrix()
        F = ms.current_mesh().face_matrix()

    n_verts = ms.current_mesh().vertex_number()
    n_faces = ms.current_mesh().face_number()
    geo_measures = ms.get_geometric_measures()
    topo_measures = ms.get_topological_measures()
    h = geo_measures['avg_edge_length']

    connected_components = topo_measures['connected_components_number']

    # trimesh watertightness
    trimesh_mesh = trimesh.Trimesh(
        vertices=np.asarray(ms.current_mesh().vertex_matrix(), float),
        faces=np.asarray(ms.current_mesh().face_matrix(), int),
        process=False
    )
    is_watertight = trimesh_mesh.is_watertight
    is_winding_consistent = trimesh_mesh.is_winding_consistent

    # trimesh volume
    vol = trimesh_mesh.volume if is_watertight else None
    area = trimesh_mesh.area

    # pymeshlab nonmanifold edges/faces
    n_nonmanifold_edges = int(topo_measures['non_two_manifold_edges'])
    n_nonmanifold_vertices = int(topo_measures['non_two_manifold_vertices'])

    print(f"[quick {i} 1/3] {n_verts} verts, {n_faces} faces, watertight: {is_watertight}, genus: {topo_measures['genus']}, wind-consistent: {is_winding_consistent}, h = {np.round(h, 3)}, components: {connected_components}\n[quick {i} 2/3] vol = {vol}, area = {area}, non-manifold edges: {n_nonmanifold_edges}/ vertices: {n_nonmanifold_vertices}\n[quick {i} 3/3] bbox: {V.min(axis=0)} to {V.max(axis=0)}")

    if flag: 
        del ms

    return {
        "n_verts": n_verts,
        "n_faces": n_faces,
        "is_watertight": is_watertight,
        "is_winding_consistent": is_winding_consistent,
        "h": h,
        "connected_components": connected_components,
        "vol": vol,
        "area": area,
        "n_nonmanifold_edges": n_nonmanifold_edges,
        "n_nonmanifold_vertices": n_nonmanifold_vertices
    }

In [7]:
mesh = meshio.read('./final_stitched.stl')
nodes = mesh.points
cells_dict = mesh.cells_dict
print(f"cells_dict: {cells_dict}")
elems = mesh.cells_dict['triangle']
print(nodes.shape)
print(elems.shape)

# quick report
m = trimesh.Trimesh(nodes, elems)
ms = ml.MeshSet()
ms.add_mesh(ml.Mesh(nodes, elems))
print("[ORIGINAL]")
quick_mesh_report(ms, i=0)

/home/bnherrerac/miniconda3/envs/fenicsx-env/lib/python3.11/site-packages/meshio/stl/_stl.py:40: RuntimeWarning: overflow encountered in scalar multiply
  if 84 + num_triangles * 50 == filesize_bytes:


ValueError: could not convert string to float: '//+'

# A: cap-only with triangle

In [ ]:
# === A) Cap-only CDT with triangle: extract loops -> 2D CDT -> mirror -> replace ===
import numpy as np, math
import trimesh, triangle as tr
from scipy.spatial import cKDTree
import pyvista as pv

AXIS_ID = {'x':0,'y':1,'z':2}

def _axis_pair(ax:str):
    a = AXIS_ID[ax]
    t = [0,1,2]; t.remove(a)
    return a, t[0], t[1]

def _snap_plane_inplace(V:np.ndarray, ax:int, value:float, tol:float):
    mask = np.isclose(V[:,ax], value, atol=tol)
    if mask.any():
        V[mask, ax] = value

def _faces_on_plane(F:np.ndarray, V:np.ndarray, ax:int, value:float, tol:float):
    on = np.isclose(V[:,ax], value, atol=tol)
    return on[F].all(axis=1), on

def _boundary_loops_from_cap(Fcap:np.ndarray, V:np.ndarray, axis:str):
    """Return closed loops (vertex index arrays), CCW in cap plane, outer-first."""
    # boundary edges of the cap: occur once within the cap
    E = np.sort(np.vstack([Fcap[:,[0,1]], Fcap[:,[1,2]], Fcap[:,[2,0]]]), axis=1)
    e, cnt = np.unique(E, axis=0, return_counts=True)
    border = e[cnt == 1]
    # adjacency
    adj = {}
    for a,b in border:
        adj.setdefault(int(a), []).append(int(b))
        adj.setdefault(int(b), []).append(int(a))
    # walk loops
    loops = []
    seen = set()
    for s in adj:
        if s in seen: continue
        cur = s; prev = None
        loop = [cur]; seen.add(cur)
        while True:
            nbrs = [n for n in adj[cur] if n != prev]
            if not nbrs: break
            nxt = nbrs[0]
            if nxt == s: break
            loop.append(nxt); seen.add(nxt)
            prev, cur = cur, nxt
        loops.append(np.array(loop, dtype=int))
    # orient CCW & outer-first (largest area)
    a,t1,t2 = _axis_pair(axis)
    ordered, areas = [], []
    for idxs in loops:
        P2 = V[idxs][:,[t1,t2]]
        area = 0.5*np.sum(P2[:,0]*np.roll(P2[:,1],-1) - P2[:,1]*np.roll(P2[:,0],-1))
        if area < 0: idxs = idxs[::-1]; area = -area
        ordered.append(idxs); areas.append(area)
    order = np.argsort(areas)[::-1]
    loops = [ordered[i] for i in order]
    return loops, np.array(areas)[order]

def _build_pslg_from_loops(loops, V, axis:str):
    a,t1,t2 = _axis_pair(axis)
    verts2 = []; segs = []; holes = []
    off = 0
    for k,idxs in enumerate(loops):
        P2 = V[idxs][:,[t1,t2]]
        n = len(idxs)
        verts2.append(P2)
        segs.extend([[off+i, off+(i+1)%n] for i in range(n)])
        if k>0: holes.append(P2.mean(0))
        off += n
    V2 = np.vstack(verts2)
    segs = np.asarray(segs, int)
    holes = (np.asarray(holes, float) if len(holes) else None)
    return V2, segs, holes

def _triangle_cdt_pslg(V2, segments, holes=None, max_area=None, quality=30):
    A = {'vertices': V2, 'segments': segments}
    if holes is not None: A['holes'] = holes
    switches = f"pq{int(quality)}D" + (f"a{max_area}" if max_area else "")
    res = tr.triangulate(A, switches)
    if 'triangles' not in res:
        raise RuntimeError("Triangle did not produce triangles—check PSLG loops")
    return res

def _pv_poly_from_mesh(V,F):
    tri = np.asarray(F, dtype=np.int64)
    cells = np.empty((len(tri),4), dtype=np.int64); cells[:,0]=3; cells[:,1:]=tri
    return pv.PolyData(np.asarray(V, float), cells.ravel())

def _pv_lines_from_loops(V, loops, color="red"):
    # build many 2-vertex segments as lines for visibility
    segs = []
    for L in loops:
        pairs = np.column_stack([L, np.roll(L,-1)])
        segs.append(pairs)
    segs = np.vstack(segs)
    # VTK line cell array: [2, i, j, 2, k, l, ...]
    cells = np.hstack([np.full((len(segs),1),2), segs]).ravel()
    return pv.PolyData(np.asarray(V, float), lines=cells)

def _pv_show_cap_debug(V,F,axis,value,tol, loops=None, title="cap"):
    a,t1,t2 = _axis_pair(axis)
    _V = V.copy(); _snap_plane_inplace(_V, a, value, tol)
    capmask,_ = _faces_on_plane(F, _V, a, value, tol)
    pd = _pv_poly_from_mesh(_V, F[~capmask])  # interior faces
    cap = _pv_poly_from_mesh(_V, F[capmask])  # old cap
    pl = pv.Plotter()
    pl.add_mesh(pd, color="lightgray", opacity=0.4, show_edges=False)
    pl.add_mesh(cap, color="lightskyblue", opacity=0.8, show_edges=True)
    if loops is not None and len(loops):
        Lpoly = _pv_lines_from_loops(_V, loops)
        pl.add_mesh(Lpoly, color="red", line_width=2)
    pl.add_text(f"{title}  ({axis}={value})", font_size=12)
    pl.show()

def _replace_caps_one_axis_triangle(nodes, elems, axis='x', h_cap_area=None, tol=1e-6, visualize=True):
    """Replace axis-min cap by 2D CDT (Triangle) and mirror same connectivity to axis-max."""
    V = nodes.copy(); F = elems.copy()
    a,t1,t2 = _axis_pair(axis)

    # ---- MIN (value=0.0) ----
    _snap_plane_inplace(V, a, 0.0, tol)
    capmask_min, on_min = _faces_on_plane(F, V, a, 0.0, tol)
    Fcap_min = F[capmask_min]
    if len(Fcap_min)==0:
        print(f"[triangle {axis}-] no faces on plane; skip")
        return V,F

    loops_min, areas = _boundary_loops_from_cap(Fcap_min, V, axis)
    print(f"[triangle {axis}-] loops={len(loops_min)}, areas={areas}")

    if visualize:
        _pv_show_cap_debug(V,F,axis,0.0,tol,loops_min, title=f"OLD cap loops ({axis}-)")

    V2, segs, holes = _build_pslg_from_loops(loops_min, V, axis)
    res = _triangle_cdt_pslg(V2, segs, holes, max_area=h_cap_area)
    V2m = res['vertices']; Tm = res['triangles']

    # boundary mapping: first len(V2) points correspond to seam loops in concat order
    seam_idx_concat = np.concatenate(loops_min)
    assert len(seam_idx_concat) == len(V2)
    # append Steiner (if any)
    n_new = len(V2m) - len(V2)
    add_xyz = np.zeros((n_new,3), float)
    add_xyz[:,a]  = 0.0
    add_xyz[:,t1] = V2m[len(V2):,0]
    add_xyz[:,t2] = V2m[len(V2):,1]
    base = len(V); V = np.vstack([V, add_xyz])
    map_min = np.arange(len(V2m), dtype=int)
    map_min[:len(V2)] = seam_idx_concat
    map_min[len(V2):] = base + np.arange(n_new)
    F_new_min = map_min[Tm]

    # replace old min-cap
    keep = ~capmask_min
    F = np.vstack([F[keep], F_new_min])

    # ---- MAX (value=1.0) ---- mirror identical 2D triangulation, pair boundary analytically
    _snap_plane_inplace(V, a, 1.0, tol)
    capmask_max,_ = _faces_on_plane(F, V, a, 1.0, tol)
    # pair boundary by (t1,t2) kd-tree
    seam_max = np.where(np.isclose(V[:,a], 1.0, atol=tol))[0]
    Pmax = V[seam_max][:,[t1,t2]]
    kdt  = cKDTree(Pmax)
    B2   = V[seam_idx_concat][:,[t1,t2]]
    d,j  = kdt.query(B2, distance_upper_bound=max(tol, 10*tol))
    if not np.isfinite(d).all():
        nmiss = np.count_nonzero(~np.isfinite(d))
        print(f"[triangle {axis}+] WARN: {nmiss} boundary verts had no pair on opposite cap within tol.")
    map_boundary_to_max = seam_max[j[np.isfinite(d)]]
    # mirror Steiner
    add_xyz2 = add_xyz.copy(); add_xyz2[:,a] = 1.0
    base2 = len(V); V = np.vstack([V, add_xyz2])
    map_max = np.empty_like(map_min)
    map_max[:len(V2)] = map_boundary_to_max
    map_max[len(V2):] = base2 + np.arange(n_new)
    F_new_max = map_max[Tm]
    keep2 = ~capmask_max
    F = np.vstack([F[keep2], F_new_max])

    if visualize:
        # visualize NEW caps (as lines only)
        loops_max = [map_boundary_to_max]  # rough overlay
        _pv_show_cap_debug(V,F,axis,0.0,tol,[], title=f"NEW cap ({axis}-)")
        _pv_show_cap_debug(V,F,axis,1.0,tol,[], title=f"NEW cap ({axis}+)")
    return V,F

def remesh_all_caps_triangle(nodes, elems, h_cap_area=None, tol=1e-6, visualize=True):
    V,F = nodes.copy(), elems.copy()
    for ax in ('x','y','z'):
        V,F = _replace_caps_one_axis_triangle(V,F, axis=ax, h_cap_area=h_cap_area, tol=tol, visualize=visualize)
    return V,F

# --- periodic seam pairing preview (for future MPC) ---
def preview_periodic_pairs(V:np.ndarray, axis:str, tol:float=1e-6):
    a,t1,t2 = _axis_pair(axis)
    Smin = np.where(np.isclose(V[:,a], 0.0, atol=tol))[0]
    Smax = np.where(np.isclose(V[:,a], 1.0, atol=tol))[0]
    kd = cKDTree(V[Smax][:,[t1,t2]])
    d,j = kd.query(V[Smin][:,[t1,t2]], distance_upper_bound=max(tol, 10*tol))
    ok = np.isfinite(d)
    n_ok = int(np.sum(ok)); n_tot = len(Smin)
    print(f"[pairs {axis}] matched {n_ok}/{n_tot} within tol={tol:g}")
    return Smin[ok], Smax[j[ok]]


In [ ]:
# Run Triangle approach (cap-only replacement)
V_tri, F_tri = remesh_all_caps_triangle(nodes, elems, h_cap_area=None, tol=1e-6, visualize=True)
m_tri = trimesh.Trimesh(V_tri, F_tri, process=False)
print("\n[Triangle caps]"); quick_mesh_report(m_tri, i=10)
# Optional healing:
# ms = ml.MeshSet(); ms.add_mesh(ml.Mesh(V_tri, F_tri)); heal(ms); quick_mesh_report(ms, i=11)

# Preview periodic pairs for MPC sanity:
preview_periodic_pairs(V_tri, 'x', tol=1e-6)
preview_periodic_pairs(V_tri, 'y', tol=1e-6)
preview_periodic_pairs(V_tri, 'z', tol=1e-6)


# B: cap-only with GMSH

In [ ]:
# === B) Cap-only with Gmsh 2D: reuse global vertex ids as gmsh point tags; mirror triangles ===
import numpy as np, math
import trimesh, pyvista as pv
import gmsh
from scipy.spatial import cKDTree

AXIS_ID = {'x':0,'y':1,'z':2}

def _axis_pair(ax:str):
    a = AXIS_ID[ax]; t=[0,1,2]; t.remove(a); return a,t[0],t[1]

def _snap_plane_inplace(V, ax, value, tol):
    m = np.isclose(V[:,ax], value, atol=tol)
    if m.any(): V[m,ax] = value

def _faces_on_plane(F, V, ax, value, tol):
    on = np.isclose(V[:,ax], value, atol=tol)
    return on[F].all(axis=1), on

def _boundary_loops_from_cap(Fcap, V, axis):
    E = np.sort(np.vstack([Fcap[:,[0,1]], Fcap[:,[1,2]], Fcap[:,[2,0]]]), axis=1)
    e,cnt = np.unique(E, axis=0, return_counts=True)
    border = e[cnt==1]
    adj = {}
    for a,b in border:
        adj.setdefault(int(a),[]).append(int(b))
        adj.setdefault(int(b),[]).append(int(a))
    loops=[]; seen=set()
    for s in adj:
        if s in seen: continue
        cur=s; prev=None
        loop=[cur]; seen.add(cur)
        while True:
            nbrs=[n for n in adj[cur] if n!=prev]
            if not nbrs: break
            nxt=nbrs[0]
            if nxt==s: break
            loop.append(nxt); seen.add(nxt)
            prev,cur=cur,nxt
        loops.append(np.array(loop,int))
    a,t1,t2 = _axis_pair(axis)
    ordered, areas = [], []
    for idxs in loops:
        P2 = V[idxs][:,[t1,t2]]
        area = 0.5*np.sum(P2[:,0]*np.roll(P2[:,1],-1) - P2[:,1]*np.roll(P2[:,0],-1))
        if area<0: idxs=idxs[::-1]; area=-area
        ordered.append(idxs); areas.append(area)
    order=np.argsort(areas)[::-1]
    return [ordered[i] for i in order], np.array(areas)[order]

def _pv_poly_from_mesh(V,F):
    tri = np.asarray(F, dtype=np.int64)
    cells = np.empty((len(tri),4), dtype=np.int64); cells[:,0]=3; cells[:,1:]=tri
    return pv.PolyData(np.asarray(V, float), cells.ravel())

def _pv_lines_from_loops(V, loops, color="red"):
    segs=[]
    for L in loops:
        segs.append(np.column_stack([L, np.roll(L,-1)]))
    segs = np.vstack(segs) if len(segs) else np.zeros((0,2),int)
    cells = np.hstack([np.full((len(segs),1),2), segs]).ravel() if len(segs) else None
    return pv.PolyData(np.asarray(V,float), lines=cells) if cells is not None else pv.PolyData()

def _pv_show_cap_debug(V,F,axis,value,tol, loops=None, title="cap"):
    a,t1,t2 = _axis_pair(axis)
    _V=V.copy(); _snap_plane_inplace(_V,a,value,tol)
    capmask,_ = _faces_on_plane(F,_V,a,value,tol)
    pd = _pv_poly_from_mesh(_V, F[~capmask])
    cap= _pv_poly_from_mesh(_V, F[capmask])
    pl = pv.Plotter()
    pl.add_mesh(pd, color="lightgray", opacity=0.4, show_edges=False)
    pl.add_mesh(cap, color="lightskyblue", opacity=0.8, show_edges=True)
    if loops:
        Lpoly = _pv_lines_from_loops(_V, loops)
        pl.add_mesh(Lpoly, color="red", line_width=2)
    pl.add_text(f"{title}  ({axis}={value})", font_size=12)
    pl.show()

def _gmsh_triangulate_cap(nodes:np.ndarray, loops:list[np.ndarray], axis='x', value=0.0, h_cap=None):
    gmsh.initialize()
    gmsh.model.add("cap2d")
    a,t1,t2 = _axis_pair(axis)
    boundary_concat = np.concatenate(loops).astype(int)

    # 1) points with tags == global ids
    for gid in boundary_concat:
        p = nodes[gid]
        xyz = [0.0,0.0,0.0]; xyz[a]=value; xyz[t1]=p[t1]; xyz[t2]=p[t2]
        gmsh.model.geo.addPoint(xyz[0], xyz[1], xyz[2], h_cap if h_cap else 0.0, tag=int(gid))

    # 2) lines for each loop
    cloop=[]
    for L in loops:
        ls=[]
        for i in range(len(L)):
            ls.append(gmsh.model.geo.addLine(int(L[i]), int(L[(i+1)%len(L)])))
        cloop.append(gmsh.model.geo.addCurveLoop(ls))

    s = gmsh.model.geo.addPlaneSurface([cloop[0]] + cloop[1:])
    gmsh.model.geo.synchronize()

    if h_cap is not None:
        gmsh.option.setNumber("Mesh.CharacteristicLengthMax", float(h_cap))
    gmsh.option.setNumber("Mesh.Algorithm", 6)  # Delaunay
    gmsh.model.mesh.generate(2)

    nodeTags, nodeCoords, _ = gmsh.model.mesh.getNodes(dim=2, tag=s)
    nodeTags = np.asarray(nodeTags, dtype=np.int64)
    nodeCoords = np.asarray(nodeCoords, dtype=float).reshape(-1,3)

    elemTypes, elemTags, elemNodeTags = gmsh.model.mesh.getElements(dim=2, tag=s)
    tri_idx = [i for i,t in enumerate(elemTypes) if t==2]
    if len(tri_idx)==0:
        gmsh.finalize()
        raise RuntimeError("No triangles generated on cap.")
    T_conn = np.asarray(elemNodeTags[tri_idx[0]], dtype=np.int64).reshape(-1,3)

    # Local indexing: boundary first (in boundary_concat order), then steiner
    tag_to_local = {int(tag): i for i, tag in enumerate(boundary_concat)}
    for tag in nodeTags:
        if int(tag) not in tag_to_local:
            tag_to_local[int(tag)] = len(tag_to_local)
    T_local = np.vectorize(lambda t: tag_to_local[int(t)], otypes=[np.int64])(T_conn)

    steiner_tags = [int(tag) for tag in nodeTags if int(tag) not in boundary_concat]
    steiner_xyz  = np.array([nodeCoords[list(nodeTags).index(st)] for st in steiner_tags], float) if steiner_tags else np.zeros((0,3))
    gmsh.finalize()
    return T_local.astype(np.int32), np.asarray(steiner_tags, int), steiner_xyz

def _replace_caps_one_axis_gmsh(nodes, elems, axis='x', tol=1e-6, h_cap=None, visualize=True):
    V = nodes.copy(); F = elems.copy()
    a,t1,t2 = _axis_pair(axis)

    # ---- MIN ----
    _snap_plane_inplace(V,a,0.0,tol)
    capmask_min, _ = _faces_on_plane(F,V,a,0.0,tol)
    Fcap_min = F[capmask_min]
    if len(Fcap_min)==0:
        print(f"[gmsh {axis}-] no faces on plane; skip"); 
        return V,F

    loops_min, areas = _boundary_loops_from_cap(Fcap_min, V, axis)
    print(f"[gmsh {axis}-] loops={len(loops_min)}, areas={areas}")
    if visualize:
        _pv_show_cap_debug(V,F,axis,0.0,tol,loops_min, title=f"OLD cap loops ({axis}-)")

    Tloc, steiner_tags, steiner_xyz = _gmsh_triangulate_cap(V, loops_min, axis=axis, value=0.0, h_cap=h_cap)
    # append steiner at x=0
    base = len(V); V = np.vstack([V, steiner_xyz])
    boundary_concat = np.concatenate(loops_min)
    nb = len(boundary_concat)
    map_min = np.empty((Tloc.max()+1,), dtype=int)
    # fill via dictionary from _gmsh_triangulate_cap local indexing:
    # the function guarantees local 0..nb-1 == boundary order, nb.. == steiner order
    map_min[:nb] = boundary_concat
    map_min[nb:] = base + np.arange(len(steiner_xyz))
    F_new_min = map_min[Tloc]
    keep = ~capmask_min
    F = np.vstack([F[keep], F_new_min])

    # ---- MAX ---- mirror exact 2D connectivity
    _snap_plane_inplace(V,a,1.0,tol)
    capmask_max,_ = _faces_on_plane(F,V,a,1.0,tol)
    seam_max = np.where(np.isclose(V[:,a], 1.0, atol=tol))[0]
    Pmax = V[seam_max][:,[t1,t2]]
    B2   = V[boundary_concat][:,[t1,t2]]
    kd   = cKDTree(Pmax)
    d,j  = kd.query(B2, distance_upper_bound=max(tol, 10*tol))
    if not np.isfinite(d).all():
        print(f"[gmsh {axis}+] WARN: {np.sum(~np.isfinite(d))} boundary verts had no pair within tol.")
    map_boundary_to_max = seam_max[j[np.isfinite(d)]]
    # mirror Steiner to x=1
    add2 = steiner_xyz.copy(); 
    if len(add2): add2[:,a] = 1.0
    base2 = len(V); V = np.vstack([V, add2])
    map_max = np.empty_like(map_min)
    map_max[:nb] = map_boundary_to_max
    map_max[nb:] = base2 + np.arange(len(add2))
    F_new_max = map_max[Tloc]
    keep2 = ~capmask_max
    F = np.vstack([F[keep2], F_new_max])

    if visualize:
        _pv_show_cap_debug(V,F,axis,0.0,tol,[], title=f"NEW cap ({axis}-)")
        _pv_show_cap_debug(V,F,axis,1.0,tol,[], title=f"NEW cap ({axis}+)")
    return V,F

def remesh_all_caps_gmsh(nodes, elems, h_cap=None, tol=1e-6, visualize=True):
    V,F = nodes.copy(), elems.copy()
    for ax in ('x','y','z'):
        V,F = _replace_caps_one_axis_gmsh(V,F, axis=ax, tol=tol, h_cap=h_cap, visualize=visualize)
    return V,F

# periodic pairs preview (same as in A)
def preview_periodic_pairs(V:np.ndarray, axis:str, tol:float=1e-6):
    a = AXIS_ID[axis]; t = [0,1,2]; t.remove(a); t1,t2 = t
    Smin = np.where(np.isclose(V[:,a], 0.0, atol=tol))[0]
    Smax = np.where(np.isclose(V[:,a], 1.0, atol=tol))[0]
    kd = cKDTree(V[Smax][:,[t1,t2]])
    d,j = kd.query(V[Smin][:,[t1,t2]], distance_upper_bound=max(tol, 10*tol))
    ok = np.isfinite(d)
    print(f"[pairs {axis}] matched {int(np.sum(ok))}/{len(Smin)} within tol={tol:g}")
    return Smin[ok], Smax[j[ok]]


In [ ]:
# Run Gmsh approach (cap-only replacement)
V_gm, F_gm = remesh_all_caps_gmsh(nodes, elems, h_cap=0.02, tol=1e-6, visualize=True)
m_gm = trimesh.Trimesh(V_gm, F_gm, process=False)
print("\n[Gmsh caps]"); quick_mesh_report(m_gm, i=20)
# Optional healing:
# ms = ml.MeshSet(); ms.add_mesh(ml.Mesh(V_gm, F_gm)); heal(ms); quick_mesh_report(ms, i=21)

# Periodic pairing preview for MPC
preview_periodic_pairs(V_gm, 'x', tol=1e-6)
preview_periodic_pairs(V_gm, 'y', tol=1e-6)
preview_periodic_pairs(V_gm, 'z', tol=1e-6)
